In [492]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup
import re

In [493]:
site = 'https://www.hanbit.co.kr/'
url = f'{site}store/books/new_book_list.html'
req = requests.get(url)
html = req.text
#html

In [494]:
soup = BeautifulSoup(html, 'html.parser')

In [495]:
lis = soup.select('div.sub_book_list_area>li')
len(lis)

20

In [496]:
li = lis[1]
href = li.find('a')['href']
href

'/store/books/look.php?p_code=B6972407557'

In [497]:
sub_url = site + href
sub_url

'https://www.hanbit.co.kr//store/books/look.php?p_code=B6972407557'

In [498]:
sub_req = requests.get(sub_url)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html, 'html.parser')

In [499]:
#제목
info = sub_soup.select_one('.store_product_info_box')
title = info.find('h3').string
title

'다시, 배우다'

In [500]:
#저자
book_lis = info.select('.info_list > li')
author = book_lis[0].find('span').get_text().strip()
author

'폴 김'

In [501]:
#역자
if len(book_lis) == 5:
    translator = '-'
else:
    translator = book_lis[1].find('span').get_text().strip()
translator

'-'

In [502]:
#출간일
date = book_lis[-4].find('span').get_text().strip()
date

'2021-12-10'

In [503]:
#페이지
page = book_lis[-3].find('span').get_text().strip().split()[0]
page = int(page)
page

304

In [504]:
#정가
payments = sub_soup.select_one('.payment_box.curr').find_all('p')
price = payments[0].select_one('.pbr').get_text().strip()
price

'16,000원'

In [505]:
#할인가
sales = payments[1].select_one('.pbr').get_text().split('(')[0]
sales

'14,400원'

In [506]:
mil = payments[2].select_one('.pbr').get_text().split()[0]
mil

'800점'

In [507]:
lines = []
for li in lis:
    book_url = li.find('a')['href']
    res = requests.get(site + book_url)
    book_soup = BeautifulSoup(res.text, 'html.parser')

    info = book_soup.select_one('.store_product_info_box')
    title = info.find('h3').string
    book_lis = info.select('.info_list > li')
    author = book_lis[0].find('span').get_text().strip()
    if len(book_lis) == 5:
        translator = None
    else:
        translator = book_lis[1].find('span').get_text().strip()
    date = book_lis[-4].find('span').get_text().strip()
    page = book_lis[-3].find('span').get_text().strip().split()[0]
    
    payments = book_soup.select_one('.payment_box.curr').find_all('p')
    price = payments[0].select_one('.pbr').get_text().strip()
    sales = payments[1].select_one('.pbr').get_text().split('(')[0]
    mileage = payments[2].select_one('.pbr').get_text().split()[0]

    lines.append([title,author,translator,date,page,price,sales,mileage])
df = pd.DataFrame(lines, columns=['제목','저자','번역','출간일','페이지','정가','판매가','마일리지'])
df

,제목,저자,번역,출간일,페이지,정가,판매가,마일리지
0,하루 한 장 초등과학 365,치바 가즈요시,"허영은 , 유우종 (감수)",2021-12-14,416,"19,800원","17,820원",990점
1,"다시, 배우다",폴 김,None,2021-12-10,304,"16,000원","14,400원",800점
2,당당한 디자인 결정을 위한 9가지 방법,톰 그리버,"김민성 , 곽서희",2021-12-10,376,"22,000원","19,800원","1,100점"
3,365 부모 말하기 연습 일력(스프링북),"박재연 , 공인영",None,2021-12-01,384,"18,000원","16,200원",900점
4,공학도를 위한 수치해석(8판),"Steven C.Chapra , Raymond P. Canale","나양 , 이승배 , 임윤철",2021-11-30,600,"36,000원","36,000원","1,080점"
5,엑셀 매크로&VBA 업무 공략집,최준선,None,2021-11-30,328,"22,000원","19,800원","1,100점"
6,데이터 과학자 되는 법,"에밀리 로빈슨 , 재클린 놀리스",이창화,2021-11-26,372,"24,000원","21,600원","1,200점"
7,돈 되는 기획,김도균,None,2021-11-26,240,"14,500원","13,050원",725점
8,빠르게 찾아 바로 적용하는 엑셀 에센스 사전 100,한은숙,None,2021-11-25,300,"18,000원","16,200원",900점
9,초등 어휘력이 공부력이다,박명선,None,2021-11-25,412,"18,000원","16,200원",900점


In [508]:
def convert_unit(s):
    return int(s.replace(',','').replace('원','').replace('점',''))

In [511]:
from tqdm.notebook import tqdm

lines = []
for page in tqdm(range(1,11)):
    url = f'{site}store/books/new_book_list.html'
    req = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    lis = soup.select('.sub_book_list')
    
    for li in lis:
        href = li.find('a')['href']
        res = requests.get(site + sub_url)
        sub_soup = BeautifulSoup(res.text, 'html.parser')

        info = sub_soup.select_one('.store_product_info_box')
        title = info.find('h3').string
        book_lis = info.select('.info_list > li')
        author = book_lis[0].find('span').get_text().strip()
        if len(book_lis) == 5:
            translator = '-'
        else:
            translator = book_lis[1].find('span').get_text().strip()
        date = book_lis[-4].find('span').get_text().strip()
        page = book_lis[-3].find('span').get_text().strip().split()[0]
        
        payments = sub_soup.select_one('.payment_box.curr').find_all('p')
        price = payments[0].select_one('.pbr').get_text().strip()
        sales = payments[1].select_one('.pbr').get_text().split('(')[0]
        mil = payments[2].select_one('.pbr').get_text().split()[0]

        lines.append([title,author,translator,date,int(page),
                      convert_unit(price),convert_unit(sales),convert_unit(mil)])
df = pd.DataFrame(lines, columns=['제목','저자','번역','출간일','페이지','정가','판매가','마일리지'])
df.tail()

  0%|          | 0/10 [00:00<?, ?it/s]

,제목,저자,번역,출간일,페이지,정가,판매가,마일리지
